# Examples of how to make queries

In [ ]:
import json
import sys
import veritas.logging
from veritas.sot import sot as veritas_sot

In [ ]:
loglevel = "INFO"

In [ ]:
veritas.logging.create_logger_environment(
    config={}, 
    cfg_loglevel=loglevel,
    cfg_loghandler=sys.stdout,
    app='example',
    uuid=None)

In [ ]:
sot = veritas_sot.Sot(url="http://127.0.0.1:8080",
                      token="_your_token_",
                      ssl_verify=False,
                      debug=True)

# Devices

## get all hosts and primary_ip

In [ ]:
devices = sot.select('hostname, primary_ip4') \
             .using('nb.devices') \
             .where()
devices

## get id, hostname and custom fields of a host
All 'custom fields' are returned because GraphQL does not offer the option of selecting just a custom field.

In [ ]:
devices = sot.select('id, hostname, cf_net') \
             .using('nb.devices') \
             .where('name=lab-02.local')
devices

## Get all hosts whose name contains the word 'local'

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('name__ic=local')
devices

## get id and hostname of multiple hosts (using logical or)

In [ ]:
devices = sot.select('id, hostname') \
             .using('nb.devices') \
             .where('name=lab-02.local or name=lab-04.local')
devices

## get id and hostname of multiple hosts (using list)
Note: You can use a list but do not use a where clause like .where('name=['xxx','yyy']'). That does **not** work

In [ ]:
devices = sot.select('id, hostname') \
             .using('nb.devices') \
             .where(name=['lab-02.local', 'lab-04.local'])
devices

## get all hosts of a location

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('location=office')
devices

## get all hosts of two locations

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('location=office or location=office')
devices

## get hosts with cf_net=testnet and platform=ios‚

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('cf_net=testnet and platform=ios')
devices

## get hosts using multiple (but different) cf_fields (or)

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('cf_net=testnet or cf_snmp_credentials=cred-1')
devices

## get all hosts with a certain role

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('role=network')
devices

## get all hosts with a certain device type

In [ ]:
devices = sot.select('hostname') \
             .using('nb.devices') \
             .where('device_type=iosv')
devices

## You can convert the data to a pandas dataframe

In [ ]:
devices = sot.select('device_id, hostname, platform, device_type') \
             .using('nb.devices') \
             .reformat('devices_as_pandas') \
             .where()
display(devices)

# Interfaces

## get all hosts with a certain interface name
The parameter 'interfaces_name_' is a **subquery** of the device

In [ ]:
devices = sot.select('hostname, interfaces') \
             .using('nb.devices') \
             .where('interfaces_name__ic=gigabit')
devices

# Prefixes

## get all prefixes with description, vlan and location

In [ ]:
prefixes = sot.select('prefix, description, vlan, location') \
             .using('nb.prefixes') \
             .where()
prefixes

## get all prefixes within a specififc range

In [ ]:
prefixes = sot.select('prefix, description') \
              .using('nb.prefixes') \
              .where('within_include=172.16.0.0/16')
prefixes

## get all prefixes within a specififc range and with a specific role

In [ ]:
prefixes = sot.select('prefix, description') \
              .using('nb.prefixes') \
              .where('within_include="172.16.0.0/16" and role=prefix_role')
prefixes

## get all prefixes with scan_prefix=True

In [ ]:
prefixes = sot.select('prefix') \
              .using('nb.prefixes') \
              .where('cf_scan_prefix=True')
prefixes

## get all prefixes within a specific range and with scan_prefix True

In [ ]:
prefixes = sot.select('prefix') \
              .using('nb.prefixes') \
              .where('within_include="172.16.0.0/16" and cf_scan_prefix=True')
prefixes

# IP addresses

## get hostname, device_type, role and primary_ip of the host with IP=192.168.0.1

In [ ]:
devices = sot.select('address, device_type, role, primary_ip4') \
             .using('nb.ipaddresses') \
             .where('address=192.168.0.1')
devices

## get address, hostname, device_type, role and primary_ip of device within prefix 192.168.0.0/24

In [ ]:
devices = sot.select('address, hostname, device_type, role, primary_ip4_for') \
             .using('nb.ipaddresses') \
             .where('prefix=192.168.0.0/24')
devices

### you can use .transform('ipaddress_to_device') to get a list of devices
To get this list you have to add 'primary_ip4_for' to the selected values

In [ ]:
devices = sot.select('hostname, address, parent, primary_ip4_for, primary_ip4') \
             .using('nb.ipaddresses') \
             .transform(['ipaddress_to_device']) \
             .where('prefix=192.168.0.0/24')
devices

## get all IP-addresses of a device

In [ ]:
devices = sot.select('address') \
             .using('nb.ipaddresses') \
             .where('device=lab-02.local')
devices

## get all hosts where the IP address is of type host

In [ ]:
devices = sot.select('hostname, primary_ip4_for') \
             .using('nb.ipaddresses') \
             .where('type__ic=host')
devices

# VLANS

## get all vlans

In [ ]:
vlans = sot.select('vid, name, location') \
           .using('nb.vlans') \
           .where()
vlans

## get all vlans of a specific location

In [ ]:
vlans = sot.select('vid, location') \
           .using('nb.vlans') \
           .where('location=office')
vlans

# General

## get ALL locations of our SOT

In [ ]:
locations = sot.select('locations') \
               .using('nb.general') \
               .where()
locations

## get ALL tags of our SOT

In [ ]:
tags = sot.select('tags') \
          .using('nb.general') \
          .where()
tags

## get dhcp tag

In [ ]:
tag = sot.select('tags') \
         .using('nb.general') \
         .where('name=dhcp')
tag

# HLDM

## get HLDM of device

In [ ]:
hldm = sot.get.hldm(device="lab-02.local")
hldm

# Join two queries
You can join two tables. The use the table name in the select statement to distinguish between the two tables.
The join parameter defines the RIGHT table of the join. The on parameter specifies the two parameter that must be equal. Use the dotted syntax to get the 'path' to your column.

In [ ]:
vlans = sot.select('vlans.vid, vlans.name, vlans.interfaces_as_tagged, devices.name, devices.platform') \
           .using('nb.vlans as vlans') \
           .join('nb.devices as devices') \
           .on('vlans.interfaces_as_tagged[0].device.id = devices.id') \
           .where('vlans.vid=100')
display(vlans)

## you can transform the result to a pandas dataframe
'transform' takes a list of strings and transforms them according to the given order.

In [ ]:
vlans = sot.select('vlans.vid, vlans.name, vlans.interfaces_as_tagged, devices.name, devices.platform') \
           .using('nb.vlans as vlans') \
           .join('nb.devices as devices') \
           .on('vlans.interfaces_as_tagged[0].device.id = devices.id') \
           .transform(['remove_id', 'to_pandas']) \
           .where('vlans.vid=100')
display(vlans)

## you can return the list of selected values by using the transformation 'values_only'

In [ ]:
vlans = sot.select('vlans.vid, vlans.name, vlans.interfaces_as_tagged, devices.name, devices.platform') \
           .using('nb.vlans as vlans') \
           .join('nb.devices as devices') \
           .on('vlans.interfaces_as_tagged[0].device.id = devices.id') \
           .transform(['remove_id', 'values_only']) \
           .where('vlans.vid=100')
display(vlans)